# LinearModel
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
const source_name = "LinearModel";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("../../TrainingAlphas/Ensemble/EnsembleInputs.ipynb");

In [ ]:
function read_recommendee_suppressed_alpha(alpha::String, split::String)
    @assert split == "all"
    if alpha in implicit_raw_alphas
        suppress = true
        content = "implicit"
    elseif alpha in ptw_raw_alphas
        suppress = true
        content = "ptw"
    else
        suppress = false
    end
    df = read_recommendee_alpha(alpha, split)
    if suppress
        seen = get_recommendee_split(content)
        p_seen = sum(df.rating[seen.item])
        ϵ = sqrt(eps(Float32))
        if 1 - p_seen > ϵ
            df.rating[seen.item] .= 0
            df.rating ./= 1 - p_seen
        end
    end
    df
end;

In [ ]:
function compute_alpha(alpha::String, content::String)
    params = read_params(alpha)
    X = []
    for alpha in params["alphas"]
        push!(X, read_recommendee_suppressed_alpha(alpha, "all").rating)
    end
    if content in ["implicit", "ptw"]
        push!(X, fill(1.0f0 / num_items(), num_items()))
    end
    X = reduce(hcat, X)
    write_recommendee_alpha(X * params["β"], alpha)
end;

In [ ]:
compute_alpha("LinearExplicit", "explicit");

In [ ]:
compute_alpha("LinearImplicit", "implicit");

In [ ]:
compute_alpha("LinearPtw", "ptw");